In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
await init_notebook(use_omicron=True)
from IPython.display import display
import asyncio
pd.options.display.max_rows = 999

In [81]:
def cut_runs(ts):
    value, start_pos, length = find_runs(ts)
    if len(value) < 2:
        return None, None
    
    return value[-2:], length[-2:]

def uturn(code, name, bars, results, frame_type):
    close = bars["close"]
    
    if len(bars) < 20:
        return
    
    # 要求最近一日站在5日线上
    if close[-1] < np.mean(close[-5:]):
        return

    # 探测反转
    prices = bars["open"] + bars["close"]
    flags, lengths = cut_runs((prices[1:]>=prices[:-1])[-10:])
    if flags is None or flags[0]:
        return
    
    ll, rl = lengths
    pos = rl

    # 涨速
    dv = close[1:] / close[:-1]
    ddv = moving_average(dv[1:] - dv[:-1], 3)[-1]
    ddv = round(ddv, 2)
    
    if ddv < 1e-3:
        return
    
    # rsi
    rsi = relative_strength_index(close)[-1]
    stats = rsiday if frame_type == FrameType.DAY else rsi30
    prsi = stats.get_proba(code, rsi)

    results.append([name, code, ddv, rl, prsi, rsi])

In [ ]:
results = await scan(uturn, 20, tm="2021-10-26", test=False)
df = pd.DataFrame(results, columns=["name", "code", "ddv", "bottom", "prsi", "rsi"])

In [80]:
df.nlargest(10,"ddv")

,name,code,ddv,bottom,prsi,rsi
18,甘肃电投,000791.XSHE,0.02,1,0.75,61.28
34,露天煤业,002128.XSHE,0.02,1,0.63,56.58
54,台海核电,002366.XSHE,0.02,1,0.37,34.06
74,领益智造,002600.XSHE,0.02,1,0.78,62.92
89,和科达,002816.XSHE,0.02,3,0.71,61.02
115,万讯自控,300112.XSHE,0.02,1,0.82,65.29
123,佳士科技,300193.XSHE,0.02,2,0.68,61.16
125,鸿利智汇,300219.XSHE,0.02,3,0.82,68.02
131,吉艾科技,300309.XSHE,0.02,2,0.77,58.62
134,天壕环境,300332.XSHE,0.02,2,0.76,63.52


In [39]:
async def wait(results:List, check_now=True):
    # 补齐前两天的rsi和prsi并显示
    end = tf.floor(arrow.now(), FrameType.DAY)
    start = tf.day_shift(end, -39)
    codes = []
    for item in results:
        name, code, ddv, skew, msr = item
        codes.append(code)
        
        sec = Security(code)
        bars = await sec.load_bars(start, end, FrameType.DAY)
        bars = bars[np.isfinite(bars["close"])]
        
        close = bars["close"]
        rsi = np.round(relative_strength_index(close, 6)[-2:], 1)
        prsi = [rsiday.get_proba(code, r) for r in rsi]
        
        item.extend([*rsi, *prsi])
        
    df = pd.DataFrame(results,columns = ["name", "code", "ddv", "skew", "msr", "rsi1", "rsi0", "prsi1", "prsi0"])
    
    print("对以下列表进行监控:")
    display(df)
    
    now = arrow.now()
    wakeup_time = []
    for tm in ["09:59", "10:29", "10:59", "11:29", "13:29", "13:59", "14:29", "14:56"]:
        hour, minute = map(int, tm.split(":"))
        wakeup_time.append(arrow.Arrow(now.year, now.month, now.day, hour, minute, tzinfo="Asia/Shanghai"))
        
    if check_now:
        wakeup_time.insert(0, arrow.now().shift(seconds=5))
    
    for tm in wakeup_time:
        if arrow.now() > tm:
            continue
            
        seconds = (tm.timestamp - arrow.now().timestamp)
        await asyncio.sleep(seconds)
        
        for code in codes:
            sec = Security(code)
            end = arrow.now().datetime
            start = tf.shift(tf.floor(end, FrameType.MIN30), -30, FrameType.MIN30)
            
            try:
                bars = await sec.load_bars(start, end, FrameType.MIN30)
            except Exception:
                continue
            bars = bars[np.isfinite(bars["close"])]
            close = bars["close"]
            
            rsi = relative_strength_index(close, 6)[-1]
            prsi = rsi30.get_proba(code, rsi)
            if (prsi and prsi < 0.1) or rsi < 20:
                text = f"{sec.display_name}出现RSI低位,概率为{prsi}[RSI:{rsi}]"
                print(text)
                say(text)
                
async def scan_and_wait():
    tm = tf.day_shift(arrow.now(),0)
    results = await scan(uturn, 20, tm="2021-10-26", test=True)
    await wait(results)
    
await scan_and_wait()

平安银行 [False  True] [1 4]
平安银行 False True 1 4 0.4
万科A [ True False] [2 2]
国华网安 [ True False] [3 7]
深振业A [ True False] [1 2]
神州高铁 [False  True] [3 1]
神州高铁 False True 3 1 0.1
中国宝安 [False  True] [1 2]
中国宝安 False True 1 2 0.2
美丽生态 [ True False] [2 8]
深物业A [ True False] [1 2]
南玻A [ True False] [1 7]
沙河股份 None None
对以下列表进行监控:


,name,code,ddv,skew,msr,rsi1,rsi0,prsi1,prsi0
0,平安银行,000001.XSHE,-0.003600,0.4,4.000000,44.1,40.4,0.36,0.29
1,神州高铁,000008.XSHE,0.002684,0.1,0.333333,35.4,33.4,0.34,0.31
2,中国宝安,000009.XSHE,0.018854,0.2,2.000000,65.5,59.2,0.78,0.69


In [ ]:
async def ddv(code, tm=None):
    if code.startswith("6"):
        code += ".XSHG"
    else:
        code += ".XSHE"
        
    sec = Security(code)
    end = arrow.get(tm).date() if tm else arrow.now().date()
    start = tf.day_shift(end, -10)
    
    bars = await sec.load_bars(start, end, FrameType.DAY)
    close = bars["close"]
    
    d1 = close[1:]/close[:-1]
    d2 = np.diff(d1)
    return np.round(d1[-5:], 2), np.round(d2[-5:], 2)

In [ ]:
await ddv("300506")

In [ ]:
await scan("2021-10-26")

In [20]:
end = arrow.get("2021-11-01").date()
start = tf.day_shift(end, -14)

payh = await Security("000001.XSHE").load_bars(start, end, FrameType.DAY)
len(payh)

15

In [30]:
ma = moving_average(payh["close"],5)

In [31]:
cut_by_msr(payh["close"][-10:] > ma[-10:])

(array([ True, False]), array([4, 4]))